In [1]:
import numpy as np
from matplotlib import pyplot as plt, animation

from nonlinear.gauss_seidel import GaussSeidel
from nonlinear.jacobi import Jacobi
from nonlinear.solve_nonlinear import FSolve
from ode.euler_schemes import ImplicitEuler
from ode.solve_ivp import solve_ivp_implicit


%load_ext autoreload
%autoreload 2

In [2]:
plt.rcParams.update({
    "text.usetex": True,
    "animation.html": "jshtml"
})

## Problem definition

In [3]:
g = 9.81
l = 1

In [4]:
def f(_: float, y: np.ndarray) -> np.array:
    return np.array([y[1], -g/l*np.sin(y[0])])

## Simulation parameters

In [5]:
kmax = 3                    # Amount of iterations
n = 100                     # Time steps
T = 4*np.pi*np.sqrt(l/g)    # Time interval
t = np.linspace(0, T, n)
scheme = 'gauss_seidel'

## Solution
### Solve dynamic iteration

In [6]:
if scheme == 'jacobi':
    solver = Jacobi(kmax)
elif scheme == 'gauss_seidel':
    solver = GaussSeidel(kmax)
elif scheme == 'fsolve':
    solver = FSolve()
else:
    raise ValueError(f"Value of option <scheme> was {scheme} but is not allowed!")

In [7]:
y0 = np.array([np.pi/6,0])
theta = solve_ivp_implicit(t, y0, ImplicitEuler(f), solver)[:,0]

In [8]:
x = l*np.sin(theta)
y = -l*np.cos(theta)

### Animate pendulum

In [9]:
%%capture

fig = plt.figure(figsize=(5,4))
ax = fig.add_subplot(autoscale_on=False, xlim=(-l, l), ylim=(-1.5*l, 0.5*l))
ax.set_aspect('equal')
ax.grid()

line, = ax.plot([], [], 'o-', lw=2)
trace, = ax.plot([], [], '.-', lw=1, ms=2)
time_template = 'time = %.1fs'
time_text = ax.text(0.05, 0.9, '', transform=ax.transAxes)

In [10]:
def animate(i):
    xi = [0, x[i]]
    yi = [0, y[i]]

    history_x = x[:i]
    history_y = y[:i]

    line.set_data(xi, yi)
    trace.set_data(history_x, history_y)
    time_text.set_text(time_template % t[i])
    return line, trace, time_text

In [11]:
animation.FuncAnimation(fig, animate, frames=theta.size, interval=T/n*1000)